In [1]:
import urllib
import pnwkit
import pandas as pd
import sqlalchemy as sq
from sqlalchemy import create_engine
from urllib.parse import quote_plus

In [2]:
#DB Connection
conFormat = urllib.parse.quote_plus(
    'Data Source Name=Franklin;' 
    'Driver={SQL Server};' 
    'Server=FRANKLIN;' 
    'Database=PnWDB;' 
    'Trusted_connection=yes;'
)
#create engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(conFormat))

# Extract

In [3]:
#Query GraphQL API using PnwKit-Py Wrapper
#Insert API Key here
kit = pnwkit.QueryKit("ENTER API KEY")

#Query first 500 of results at once
#Retrieves only fields attributes specified in .query() method
nation_query = kit.query(
    'nations', {'first': 500}, 'id', 'nation_name', 'num_cities', 'color', 'score', 'alliance_id', 'alliance_position', 
    'soldiers', 'tanks', 'aircraft', 'ships', 'vacation_mode_turns', 'last_active'
)
#Query alliance field and attributes
alliance_query = kit.query(
    'alliances', {'first': 100}, 'id', 'name'
)
#Query treaty field and attributes
treaty_query = kit.query(
    'treaties', {'first': 500}, 'id', 'alliance1_id', 'alliance2_id', 'treaty_type'
)

#Paginate queries for all available results in each field
nations = nation_query.paginate("nations")
alliances = alliance_query.paginate("alliances")
treaties = treaty_query.paginate("treaties")

#Initialize empty dictionaries that will later be converted into Dataframes
#Also set column order here for dynamic tables
nation_data = {
    'id': [], 'alliance_id': [], 'alliance_position': [], 'color': [], 'nation_name': [], 'num_cities': [], 'score': [], 
    'vacation_mode_turns': [], 'last_active': []
}

military_data = {
    'id': [], 'soldiers': [], 'tanks': [], 'aircraft': [], 'ships': []
}

alliance_data = {
    'id': [], 'name': []
}

treaty_data = {
    'id': [], 'alliance1_id': [], 'alliance2_id': [], 'treaty_type': []
}

#static data tables
color_data = {
    'color_id': [*range(0, 16)], 'color': ['beige', 'gray', 'lime', 'green', 'white', 'brown', 'maroon', 'purple', 'blue', 
                                           'red', 'orange', 'olive', 'aqua', 'black', 'yellow', 'pink']
}

position_data = {
    'position_id': [*range(0, 6)], 'position': ['NOALLIANCE', 'APPLICANT', 'MEMBER', 'OFFICER', 'HEIR', 'LEADER']
}

In [4]:
#Convert paginator objects to dictionaries
#Iterate through paginator object: nations
for nation in nations:
    #iterate through empty dict and append values
    for key in nation_data:
        if key != 'alliance_position':
            nation_data[str(key)].append(nation.__getattribute__(key))
        #Special case where key != object atrribute
        else:
            #nested attribute alliance position
            nation_data['alliance_position'].append(nation.alliance_position.value)
    #iterate through empty dict and append values
    for key in military_data:
            military_data[str(key)].append(nation.__getattribute__(key))

#Iterate through paginator object: alliances
for alliance in alliances:
    #iterate through empty dict and append values
    for key in alliance_data:
            alliance_data[str(key)].append(alliance.__getattribute__(key))

#Iterate through paginator object: treaties
for treaty in treaties:
    #iterate through empty dict and append values
    for key in treaty_data:
        treaty_data[str(key)].append(treaty.__getattribute__(key))

# Transform

In [5]:
#create DF from dictionaries
alliances_df = pd.DataFrame(alliance_data)
#change column headers to more approprate names
alliances_df.rename(columns={'id': 'alliance_id', 'name': 'alliance_name'}, inplace=True)
alliances_df.head()

,alliance_id,alliance_name
0,790,Rose
1,1584,The Knights Radiant
2,5875,The Immortals
3,1023,The Fighting Pacifists
4,7450,Eclipse


In [6]:
#create DF from dictionaries
alliance_positions_df = pd.DataFrame(position_data)
alliance_positions_df.head()

,position_id,position
0,0,NOALLIANCE
1,1,APPLICANT
2,2,MEMBER
3,3,OFFICER
4,4,HEIR


In [7]:
#create DF from dictionaries
colors_df = pd.DataFrame(color_data)
colors_df.head()

,color_id,color
0,0,beige
1,1,gray
2,2,lime
3,3,green
4,4,white


In [8]:
#create DF from dictionaries
nations_df = pd.DataFrame(nation_data)
nations_df.head()

,id,alliance_id,alliance_position,color,nation_name,num_cities,score,vacation_mode_turns,last_active
0,5270,11515,5,pink,Belleroph,40,7445.00,0,2023-07-01 03:39:06+00:00
1,439,7450,2,brown,Howell,32,5835.00,0,2023-06-30 00:21:19+00:00
2,300,3339,2,green,BEPU,47,9571.10,0,2023-06-30 05:55:03+00:00
3,405,3339,2,green,Gladius,43,8761.25,0,2023-06-27 03:21:25+00:00
4,6,8900,5,gray,Mountania,5,390.00,0,2023-07-01 01:00:08+00:00


In [9]:
#replaces text with numbers
nations_df['color'] = nations_df['color'].replace(['beige', 'gray', 'lime', 'green', 'white', 'brown', 'maroon', 'purple', 
                                                   'blue', 'red', 'orange', 'olive', 'aqua', 'black', 'yellow', 'pink'], 
                                                  [*range(0, 16)])
#change dtypes to appropriate datetime format
nations_df['last_active'] = pd.to_datetime(nations_df.last_active).dt.tz_localize(None)
#change column headers to more approprate names
nations_df.rename(columns={'id': 'nation_id', 'num_cities': 'cities', 'color': 'color_id', 
                           'alliance_position': 'position_id', 'score': 'nation_score'}, inplace=True)
#select records in nations_df if the alliance id exists in the alliance df
new_nations_df = nations_df[nations_df['alliance_id'].isin(alliances_df['alliance_id'])]
new_nations_df.head()

,nation_id,alliance_id,position_id,color_id,nation_name,cities,nation_score,vacation_mode_turns,last_active
0,5270,11515,5,15,Belleroph,40,7445.00,0,2023-07-01 03:39:06
1,439,7450,2,5,Howell,32,5835.00,0,2023-06-30 00:21:19
2,300,3339,2,3,BEPU,47,9571.10,0,2023-06-30 05:55:03
3,405,3339,2,3,Gladius,43,8761.25,0,2023-06-27 03:21:25
4,6,8900,5,1,Mountania,5,390.00,0,2023-07-01 01:00:08


In [10]:
#create DF from dictionaries
militaries_df = pd.DataFrame(military_data)
#change column headers to more approprate names
militaries_df.rename(columns={'id': 'nation_id'}, inplace=True)
#select records in militaries_df if the alliance id exists in the alliance df
new_militaries_df = militaries_df[militaries_df['nation_id'].isin(new_nations_df['nation_id'])]
new_militaries_df.head()

,nation_id,soldiers,tanks,aircraft,ships
0,5270,0,20000,3000,0
1,439,0,16000,2400,0
2,300,189000,33000,2760,395
3,405,165000,28750,2355,305
4,6,0,0,0,0


In [11]:
#create DF from dictionaries
treaties_df = pd.DataFrame(treaty_data)
treaties_df.head()

,id,alliance1_id,alliance2_id,treaty_type
0,19893,10455,4271,Protectorate
1,20013,7817,4648,Protectorate
2,20281,2510,1584,Protectorate
3,20704,8819,3427,ODoAP
4,20773,9432,790,Protectorate


In [12]:
#change column headers to more approprate names
treaties_df.rename(columns={'id': 'treaty_id', 'alliance1_id': 'sending_alliance_id', 
                            'alliance2_id': 'receiving_alliance_id'}, inplace=True)
#select records in treaties_df if the alliance id exists in the alliance df
send_treaties_df = treaties_df[treaties_df['sending_alliance_id'].isin(alliances_df['alliance_id'])]
new_treaties_df = send_treaties_df[send_treaties_df['receiving_alliance_id'].isin(alliances_df['alliance_id'])]
new_treaties_df.head()

,treaty_id,sending_alliance_id,receiving_alliance_id,treaty_type
0,19893,10455,4271,Protectorate
1,20013,7817,4648,Protectorate
2,20281,2510,1584,Protectorate
3,20704,8819,3427,ODoAP
4,20773,9432,790,Protectorate


# Load into DB

In [13]:
with engine.begin() as start:
    start.execute('DELETE FROM treaties')
    start.execute('DELETE FROM nations')
    start.execute('DELETE FROM alliance_positions')
    start.execute('DELETE FROM color_blocs')
    start.execute('DELETE FROM militaries')
    start.execute('DELETE FROM alliances')

In [14]:
#send tables into DB, preserving column datatypes
alliances_df.to_sql('alliances', con=engine, schema='dbo', if_exists='append', index=False,
                    dtype={'alliance_id': sq.types.SMALLINT(),'alliance_name': sq.types.VARCHAR()})

-1

In [15]:
#send tables into DB, preserving column datatypes
alliance_positions_df.to_sql('alliance_positions', con=engine, schema='dbo', if_exists='append', index=False, 
                             dtype={'position_id': sq.types.SMALLINT(), 'position': sq.types.VARCHAR(50)})

-1

In [16]:
#send tables into DB, preserving column datatypes
colors_df.to_sql('color_blocs', con=engine, schema='dbo', if_exists='append', index=False, 
                 dtype={'color_id': sq.types.SMALLINT(), 'color': sq.types.VARCHAR(50)})

-1

In [17]:
#send tables into DB, preserving column datatypes
new_nations_df.to_sql('nations', con=engine, schema='dbo', if_exists='append', index=False,
                      dtype={'nation_id': sq.types.INT(), 'nation_name': sq.types.VARCHAR(100), 
                             'leader_name': sq.types.VARCHAR(100), 'cities': sq.types.SMALLINT(), 
                             'color_bloc': sq.types.VARCHAR(100), 'nation_score': sq.types.FLOAT, 
                             'alliance_id': sq.types.SMALLINT(), 'alliance_position': sq.types.VARCHAR(50),
                             'vacation_mode_turns':sq.types.INT, 'last_active': sq.types.DATETIME()})

-1

In [18]:
#send tables into DB, preserving column datatypes
new_militaries_df.to_sql('militaries', con=engine, schema='dbo', if_exists='append', index=False, 
                         dtype={'nation_id': sq.types.INT(), 'soldiers': sq.types.INT(), 'tanks': sq.types.INT(), 
                                'aircraft': sq.types.SMALLINT(), 'ships': sq.types.SMALLINT()})

-1

In [19]:
#send tables into DB, preserving column datatypes
new_treaties_df.to_sql('treaties', con=engine, schema='dbo', if_exists='append', index=False,
                       dtype={'treaty_id': sq.types.INT(), 'treaty_type': sq.types.VARCHAR(), 
                              'alliance_id': sq.types.SMALLINT(), 'turns_left': sq.types.SMALLINT()})

-1